In [1]:
# pip install -e ../

In [1]:
%env JAX_PLATFORMS=cpu

env: JAX_PLATFORMS=cpu


In [26]:
import warnings

warnings.filterwarnings("ignore")

from hydra import compose, initialize

from jumanji.training.train import train

In [27]:
env = "sudoku"  # @param ['bin_pack', 'cleaner', 'connector', 'cvrp', 'game_2048', 'job_shop', 'knapsack', 'maze', 'minesweeper', 'rubiks_cube', 'snake', "sudoku", 'tsp']
agent = "a2c"  # @param ['random', 'a2c']

In [28]:
with initialize(version_base=None, config_path="../jumanji/training/configs"):
    cfg = compose(
        config_name="config.yaml",
        overrides=[
            f"env={env}",
            f"agent={agent}",
            "logger.type=terminal",
            "logger.save_checkpoint=true",
        ],
    )
cfg

{'agent': 'a2c', 'seed': 0, 'logger': {'type': 'terminal', 'save_checkpoint': True, 'name': '${agent}_${env.name}'}, 'env': {'name': 'sudoku', 'registered_version': 'Sudoku-v0', 'network': {'num_channels': 32, 'policy_layers': [64, 64], 'value_layers': [128, 128]}, 'training': {'num_epochs': 100, 'num_learner_steps_per_epoch': 500, 'n_steps': 20, 'total_batch_size': 128}, 'evaluation': {'eval_total_batch_size': 200, 'greedy_eval_total_batch_size': 200}, 'a2c': {'normalize_advantage': False, 'discount_factor': 0.997, 'bootstrapping_factor': 0.95, 'l_pg': 1.0, 'l_td': 1.0, 'l_en': 0.01, 'learning_rate': 0.0004}}}

In [29]:
train(cfg)

INFO:root:agent: a2c
seed: 0
logger:
  type: terminal
  save_checkpoint: true
  name: ${agent}_${env.name}
env:
  name: sudoku
  registered_version: Sudoku-v0
  network:
    num_channels: 32
    policy_layers:
    - 64
    - 64
    value_layers:
    - 128
    - 128
  training:
    num_epochs: 100
    num_learner_steps_per_epoch: 500
    n_steps: 20
    total_batch_size: 128
  evaluation:
    eval_total_batch_size: 200
    greedy_eval_total_batch_size: 200
  a2c:
    normalize_advantage: false
    discount_factor: 0.997
    bootstrapping_factor: 0.95
    l_pg: 1.0
    l_td: 1.0
    l_en: 0.01
    learning_rate: 0.0004

INFO:root:{'devices': [CpuDevice(id=0)]}
INFO:root:Experiment: a2c_sudoku.


board (1, 9, 9)
embedding (1, 800)
(1, 9, 9, 9)
board (1, 9, 9)
embedding (1, 800)
(1, 9, 9, 9)


INFO:root:Starting logger.


board (1, 9, 9)
embedding (1, 800)
(1, 9, 9, 9)
Traced<ShapedArray(int8[9,9,9])>with<DynamicJaxprTrace(level=2/1)>
Traced<ShapedArray(int32[3])>with<DynamicJaxprTrace(level=2/1)>
Traced<ShapedArray(bool[])>with<DynamicJaxprTrace(level=2/1)> Traced<ShapedArray(bool[])>with<DynamicJaxprTrace(level=2/1)> Traced<ShapedArray(bool[])>with<DynamicJaxprTrace(level=2/1)>


INFO:root:Eval Stochastic >> Env Steps: 0.00e+00 | Episode Length: 31.815 | Episode Return: 0.000 | Time: 1.501


...

board (128, 9, 9)
embedding (128, 800)
(128, 9, 9, 9)
Traced<ShapedArray(int8[9,9,9])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(int8[128,9,9,9])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0
Traced<ShapedArray(int32[3])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(int32[128,3])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0
Traced<ShapedArray(bool[])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(bool[128])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0 Traced<ShapedArray(bool[])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(bool[128])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0 Traced<ShapedArray(bool[])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(bool[128])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0
board (128, 9, 9)
embedding (128, 800)
(128, 9, 9, 9)


INFO:root:Train >> Env Steps: 0.00e+00 | Advantage: -0.000 | Critic Loss: 0.000 | Entropy: 3.029 | Entropy Loss: -3.029 | Policy Loss: -0.000 | Steps Per Second: 4,394 | Time: 291.278 | Total Loss: -0.030 | Value: 0.000
INFO:root:Eval Stochastic >> Env Steps: 1.28e+06 | Episode Length: 31.475 | Episode Return: 0.000 | Time: 0.211
INFO:root:Eval Greedy >> Env Steps: 1.28e+06 | Episode Length: 32.000 | Episode Return: 0.000 | Time: 0.001


board (128, 9, 9)
embedding (128, 800)
(128, 9, 9, 9)
Traced<ShapedArray(int8[9,9,9])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(int8[128,9,9,9])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0
Traced<ShapedArray(int32[3])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(int32[128,3])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0
Traced<ShapedArray(bool[])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(bool[128])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0 Traced<ShapedArray(bool[])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(bool[128])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0 Traced<ShapedArray(bool[])>with<BatchTrace(level=5/1)> with
  val = Traced<ShapedArray(bool[128])>with<DynamicJaxprTrace(level=4/1)>
  batch_dim = 0
board (128, 9, 9)
embedding (128, 800)
(128, 9, 9, 9)


INFO:root:Saving checkpoint...
INFO:root:Checkpoint saved at 'training_state'.
INFO:root:Closing logger...


KeyboardInterrupt: 